In [29]:
from datasets import load_dataset
import pandas as pd
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer, TrainingArguments, Trainer
from torch.utils.data import DataLoader
#from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

In [2]:
dataset = load_dataset("Gustavosta/Stable-Diffusion-Prompts")

Using custom data configuration Gustavosta--Stable-Diffusion-Prompts-f4211d2c5626deea
Found cached dataset parquet (F:/MachineLearning/.cache/hugging_face/datasets/Gustavosta___parquet/Gustavosta--Stable-Diffusion-Prompts-f4211d2c5626deea/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
train_dataset = dataset['train']
valid_dataset = dataset['test']

In [34]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [26]:
pd.DataFrame(train_dataset).head()

,Prompt
0,realistic car 3 d render sci - fi car and sci ...
1,a comic potrait of a female necromamcer with b...
2,"steampunk market interior, colorful, 3 d scene..."
3,"“A portrait of a cyborg in a golden suit, D&D ..."
4,A full portrait of a beautiful post apocalypti...


In [5]:
# Tokenize the training set
train_encodings = tokenizer(dataset['train']['Prompt'], truncation=True, padding=True)

In [36]:
model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device).train()

In [31]:
device = 'cuda'

In [41]:
# Define the training arguments
num_train_epochs = 3
per_device_train_batch_size = 8
warmup_steps = 500
gradient_accumulation_steps = 16
learning_rate = 1e-4

# Define the optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=len(train_encodings)//per_device_train_batch_size+1, epochs=num_train_epochs, pct_start=0.05)

# Define the data loader
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_encodings['attention_mask']))
train_loader = DataLoader(train_dataset, batch_size=per_device_train_batch_size, shuffle=True)

# Start the training loop
for epoch in range(num_train_epochs):
    model.train()
    train_loss = 0
    for i, batch in enumerate(train_loader):
        # Prepare the inputs and targets
        input_ids, attention_mask = batch
        target_ids = input_ids.clone()

        # Move the inputs and targets to the GPU
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        target_ids = target_ids.to(device)

        # Zero out the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids)

        # Compute the loss
        loss = outputs.loss / gradient_accumulation_steps

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()
        scheduler.step()

        # Accumulate the training loss
        train_loss += loss.item()

        # Log the training loss
        if (i+1) % gradient_accumulation_steps == 0:
            avg_train_loss = train_loss / gradient_accumulation_steps
            print(f"Epoch {epoch+1}, Batch {i+1}, Train Loss: {avg_train_loss:.4f}")
            train_loss = 0

# Save the trained model
model.save_pretrained('./t5-small-finetuned')

RuntimeError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 6.00 GiB total capacity; 5.26 GiB already allocated; 0 bytes free; 5.38 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [13]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=12,  # batch size per device during training
    per_device_eval_batch_size=12,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # log every n updates steps
    save_total_limit=1,              # save only one checkpoint
    gradient_accumulation_steps=4,   # gradient accumulation steps
    learning_rate=1e-4               # learning rate
)

In [27]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("Gustavosta/Stable-Diffusion-Prompts")

# Load the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Tokenize the training set
train_encodings = tokenizer(dataset['train']['Prompt'], truncation=True, padding=True)

# Define the T5 model and set it to training mode
model = T5ForConditionalGeneration.from_pretrained('t5-small').train()

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # log every n updates steps
    save_total_limit=1,              # save only one checkpoint
    gradient_accumulation_steps=4,   # gradient accumulation steps
    learning_rate=1e-4,              # learning rate
    seed=42                          # random seed for reproducibility
)

# Define the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    use_legacy_dataset=False
)

# Fine-tune the model
trainer.train()


Using custom data configuration Gustavosta--Stable-Diffusion-Prompts-f4211d2c5626deea
Found cached dataset parquet (F:/MachineLearning/.cache/hugging_face/datasets/Gustavosta___parquet/Gustavosta--Stable-Diffusion-Prompts-f4211d2c5626deea/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

TypeError: __init__() got an unexpected keyword argument 'use_legacy_dataset'

In [17]:
import tensorflow as tf

In [20]:
import tensorflow as tf

print(tf.__version__)


AttributeError: module 'tensorflow' has no attribute '__version__'

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    use_legacy_dataset=False
)


TypeError: __init__() got an unexpected keyword argument 'use_legacy_dataset'

In [12]:
# Tokenize the input and target texts
def tokenize_function(example):
    inputs = example['prompt']
    targets = example['text']
    input_ids = tokenizer.encode(inputs, padding='max_length', max_length=512, truncation=True)
    target_ids = tokenizer.encode(targets, padding='max_length', max_length=512, truncation=True)
    return {'input_ids': input_ids, 'attention_mask': [1]*len(input_ids), 'decoder_input_ids': target_ids[:-1], 'decoder_attention_mask': [1]*len(target_ids[:-1]), 'labels': target_ids[1:]}

tokenized_dataset = dataset.map(tokenize_function, batched=True)

  0%|          | 0/74 [00:00<?, ?ba/s]

KeyError: 'prompt'

In [16]:
class DatasetPrep:
    def __init__(self):
        self.tokenizer = None

    def set_tokenzier(self, tokenizer):
        self.tokenizer = tokenizer

    def add_eos_to_examples(self, example):
        example['input_text'] = 'question: %s  context: %s </s>' % (example['question'], example['context'])
        example['target_text'] = '%s </s>' % example['answers']
        return example

    # tokenize the examples
    def convert_to_features(self, example_batch):
        input_encodings = self.tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True,
                                                           max_length=512)
        target_encodings = self.tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True,
                                                            max_length=16)

        encodings = {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'target_ids': target_encodings['input_ids'],
            'target_attention_mask': target_encodings['attention_mask']
        }

        return encodings
    
    def prepare_dataset(self):
        train_dataset = train_dataset.map(self.add_eos_to_examples)
        # map convert_to_features batch wise
        train_dataset = train_dataset.map(self.convert_to_features, batched=True)

        valid_dataset = valid_dataset.map(self.add_eos_to_examples, load_from_cache_file=False)
        valid_dataset = valid_dataset.map(self.convert_to_features, batched=True, load_from_cache_file=False)

        # set the tensor type and the columns which the dataset should return
        columns = ['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask']
        train_dataset.set_format(type='torch', columns=columns)
        valid_dataset.set_format(type='torch', columns=columns)

        # cache the dataset, so we can load it directly for training

        torch.save(train_dataset, 'train_data.pt')
        torch.save(valid_dataset, 'valid_data.pt')

In [17]:

obj = DatasetPrep()
obj.prepare_dataset()

UnboundLocalError: local variable 'train_dataset' referenced before assignment